In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, f1_score,roc_curve, precision_score, recall_score,roc_auc_score
from sklearn import linear_model, tree, ensemble
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action="ignore")

In [65]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    return df

In [20]:
class Preprocessing:
    def __init__(self, df):
        self.df = df
        
    def shape(self):
        print(f'shape: {self.df.shape}')
    
    def dtypes(self, pr=False):
        print("Types")
        if pr:
            print(self.df.dtypes)
            
    def supposed2beint(self):
        float_cols = [column for column in self.df.columns if self.df[column].dtype == 'float']
        int2be_cols = []
        for col in float_cols:
            if (self.df[col].fillna(-9999) % 1  == 0).all() == True:
                int2be_cols.append(col)
        return int2be_cols
                
    def isNaN(self, pr=False):
        if pr:
            print("Contain NaN")
            print(self.df.isnull().sum())
        else:
            return self.df.columns[self.df.isna().any()].tolist()
    
    def isObject(self):
        return [column for column in self.df.columns if self.df[column].dtype == 'object']
        
    def check_dataframe(self):
        self.shape()
        self.dtypes(True)
        self.isNaN(True)
        
    def fillNaN(self):
        nan_cols = self.isNaN()
        int2be_cols = self.supposed2beint()
        # nan_cols == int2be_cols they are the same
        for col in int2be_cols:
            self.df[col].fillna(round(self.df[col].mean()), inplace=True)
            self.df[col] = self.df[col].astype(int)
            
    def adjust_category_cols(self):
        self.fillNaN()
        #PreferredLoginDevice
        self.df.loc[self.df["PreferredLoginDevice"] == "Mobile Phone", "PreferredLoginDevice"] = "Phone"
        #PreferredPaymentMode
        self.df.loc[self.df["PreferredPaymentMode"] == "Credit Card", "PreferredPaymentMode"] = "CC"
        self.df.loc[self.df["PreferredPaymentMode"] == "Cash on Delivery", "PreferredPaymentMode"] = "COD"
        #PreferredLoginDevice
        self.df.loc[self.df["PreferredLoginDevice"] == "Mobile Phone", "PreferredLoginDevice"] = "Phone"
        
    def drop_useless_cols(self):
        self.df.drop(['CustomerID'], axis=1, inplace=True)
        
    def split_target(self):
        self.adjust_category_cols()
        self.drop_useless_cols()
        self.X = self.df.drop('Churn', axis=1)
        self.y = self.df['Churn'].astype(int).to_numpy()
        
    def find_enc_method(self):
        cat_cols = self.isObject()
        one_hot_cols = [col for col in cat_cols if self.X[col].nunique() <=3]
        label_enc_cols = [col for col in cat_cols if col not in one_hot_cols]
        return one_hot_cols, label_enc_cols, cat_cols
    
    def encoding(self):
        one_hot_cols, label_enc_cols, cat_cols = self.find_enc_method()
        num_cols = [col for col in self.X.columns if col not in cat_cols]
        X_OHE, X_LE, X_NUM = self.X[one_hot_cols].copy(), self.X[label_enc_cols].copy(), self.X[num_cols].copy()
        self.OHE = OneHotEncoder(drop='first', handle_unknown='error')
        X_OHE = self.OHE.fit_transform(X_OHE).toarray()
        self.le_dict = {}
        self.LE = LabelEncoder()
#         X_LE[label_enc_cols] = X_LE[label_enc_cols].apply(lambda col: self.LE.fit_transform(col))   
        for col in X_LE.columns:
            self.le_dict[col] = self.LE.fit(X_LE[col])
            X_LE[col] = self.le_dict[col].transform(X_LE[col])
        return X_OHE, X_LE.to_numpy(), X_NUM.to_numpy()

    def scaling(self):
        X_OHE, X_LE, X_num = self.encoding()
        self.SS = StandardScaler()
        X_num = self.SS.fit_transform(X_num)
        self.X_total = np.concatenate((X_OHE, X_LE, X_num), axis=1)
#         self.X_total = self.SS.fit_transform(self.X_total)
        
    def get_encoders(self):
        return self.OHE, self.LE, self.le_dict
    
    def get_scaler(self):
        return self.SS   

    def get_default_Xy(self):
        self.split_target()
        return self.X, self.y
      
    def get_Xy(self):
        self.split_target()
        self.scaling()
        return self.X_total, self.y

df = pd.read_excel('/kaggle/input/ecommerce-customer-churn-analysis-and-prediction/E Commerce Dataset.xlsx', sheet_name='E Comm')
data = pd.read_excel('/kaggle/input/ecommerce-customer-churn-analysis-and-prediction/E Commerce Dataset.xlsx', sheet_name='Data Dict')
pre = Preprocessing(df)
X, y = pre.get_Xy()

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
model = linear_model.LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
CR = classification_report(y_test, y_pred)
CM = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
print("="*60)
print(f"Accuracy: {accuracy:.4f}")
print("="*60)
print(f"Classification Report:\n {CR}")
print("="*60)
print(f"Confusion Matrix: {CM}")
print("="*60)
print(f"F1 Score: {f1:.4f}")
print("="*60)
print(f"ROC AUC Score: {roc_auc:.4f}")
print("="*60)

Accuracy: 0.8757
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.96      0.93       936
           1       0.70      0.46      0.55       190

    accuracy                           0.88      1126
   macro avg       0.80      0.71      0.74      1126
weighted avg       0.86      0.88      0.86      1126

Confusion Matrix: [[899  37]
 [103  87]]
F1 Score: 0.5541
ROC AUC Score: 0.7092


In [36]:
import optuna
from xgboost import XGBClassifier
def objective_xgb(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_categorical("n_estimators", [150, 200, 300, 3000]),
        'max_depth': trial.suggest_categorical('max_depth', [4,5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = XGBClassifier(**param)
    model.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)], verbose=False)   
    preds = model.predict(X_test)
    
    acc = accuracy_score(y_test, preds)      

    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective_xgb, n_trials=50)
params_xgb = study.best_trial.params
print('Number of finished trials:', len(study.trials))
print('Best trial:', params_xgb)

[I 2023-03-29 14:32:06,327] A new study created in memory with name: no-name-8c0fbc04-a5c0-4841-b0c3-fd4a937ea880
[I 2023-03-29 14:32:07,212] Trial 0 finished with value: 0.837181764357608 and parameters: {'lambda': 0.03124640759785608, 'alpha': 0.0024235868387922437, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.018, 'n_estimators': 150, 'max_depth': 5, 'min_child_weight': 279}. Best is trial 0 with value: 0.837181764357608.
[I 2023-03-29 14:32:08,434] Trial 1 finished with value: 0.837181764357608 and parameters: {'lambda': 0.02321929076288543, 'alpha': 0.007059116293567424, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.012, 'n_estimators': 200, 'max_depth': 5, 'min_child_weight': 189}. Best is trial 0 with value: 0.837181764357608.
[I 2023-03-29 14:32:09,371] Trial 2 finished with value: 0.84251036116045 and parameters: {'lambda': 0.20396806019603347, 'alpha': 0.0015175872497201436, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.016, 'n

Number of finished trials: 50
Best trial: {'lambda': 6.863193244163752, 'alpha': 0.9369522155311274, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.016, 'n_estimators': 3000, 'max_depth': 5, 'min_child_weight': 1}


In [46]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
model = XGBClassifier(**params_xgb)
model.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)], verbose=False)   
y_pred = model.predict(X_test)
print(f"Accuracy: {round(accuracy_score(y_pred, y_test), 2)}")
print("="*60)
print(f"Recall: {round(recall_score(y_pred,y_test),2)}")
print("="*60)
print(f"Precision: {round(precision_score(y_pred,y_test), 2)}")
print("="*60)
print(f"F1: {round(f1_score(y_pred,y_test), 2)}")
print("="*60)
print(f"Auc: {round(roc_auc_score(y_pred,y_test), 2)}")
print("="*60)
CR = classification_report(y_test, y_pred)
CM = confusion_matrix(y_test, y_pred)
print(f"Classification Report:\n {CR}")
print("="*60)
print(f"Confusion Matrix: {CM}")
print("="*60)

Accuracy: 0.96
Recall: 0.92
Precision: 0.85
F1: 0.88
Auc: 0.94
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98      1405
           1       0.92      0.85      0.88       284

    accuracy                           0.96      1689
   macro avg       0.94      0.91      0.93      1689
weighted avg       0.96      0.96      0.96      1689

Confusion Matrix: [[1383   22]
 [  44  240]]


In [47]:
import optuna
from lightgbm import LGBMClassifier
def objective_lgbm(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    param = {
        'random_state': 42,
        'n_estimators': trial.suggest_categorical("n_estimators", [150, 200, 300, 3000]),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = LGBMClassifier(**param)
    model.fit(X_train, y_train, early_stopping_rounds=100, eval_set=[(X_test, y_test)], verbose=False)    
    preds = model.predict(X_test)
    
    acc = accuracy_score(y_test, preds)      

    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective_lgbm, n_trials=50)
params_lgbm = study.best_trial.params
print('Number of finished trials:', len(study.trials))
print('Best trial:', params_lgbm)

[I 2023-03-29 15:06:53,978] A new study created in memory with name: no-name-bfd52714-127c-47e2-aba9-40163b07b894
[I 2023-03-29 15:07:00,438] Trial 0 finished with value: 0.8981645944345766 and parameters: {'n_estimators': 3000, 'reg_alpha': 8.440513826558606, 'reg_lambda': 1.243054388693692, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 10, 'num_leaves': 175, 'min_child_samples': 206, 'min_data_per_groups': 37}. Best is trial 0 with value: 0.8981645944345766.
[I 2023-03-29 15:07:28,830] Trial 1 finished with value: 0.9413854351687388 and parameters: {'n_estimators': 3000, 'reg_alpha': 0.00527721749672363, 'reg_lambda': 1.0253900929023407, 'colsample_bytree': 0.3, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 10, 'num_leaves': 813, 'min_child_samples': 133, 'min_data_per_groups': 73}. Best is trial 1 with value: 0.9413854351687388.
[I 2023-03-29 15:08:10,316] Trial 2 finished with value: 0.9715808170515098 and parameters: {'n_estimators': 3000

Number of finished trials: 50
Best trial: {'n_estimators': 3000, 'reg_alpha': 0.047353474470774884, 'reg_lambda': 0.08826733377969398, 'colsample_bytree': 0.6, 'subsample': 1.0, 'learning_rate': 0.017, 'max_depth': 20, 'num_leaves': 982, 'min_child_samples': 38, 'min_data_per_groups': 14}


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
model = LGBMClassifier(**params_lgbm)
model.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)], verbose=False)   
y_pred = model.predict(X_test)
print(f"Accuracy: {round(accuracy_score(y_pred, y_test), 2)}")
print("="*60)
print(f"Recall: {round(recall_score(y_pred,y_test),2)}")
print("="*60)
print(f"Precision: {round(precision_score(y_pred,y_test), 2)}")
print("="*60)
print(f"F1: {round(f1_score(y_pred,y_test), 2)}")
print("="*60)
print(f"Auc: {round(roc_auc_score(y_pred,y_test), 2)}")
print("="*60)
CR = classification_report(y_test, y_pred)
CM = confusion_matrix(y_test, y_pred)
print(f"Classification Report:\n {CR}")
print("="*60)
print(f"Confusion Matrix: {CM}")
print("="*60)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
Accuracy: 0.98
Recall: 0.96
Precision: 0.9
F1: 0.93
Auc: 0.97
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      1405
           1       0.96      0.90      0.93       284

    accuracy                           0.98      1689
   macro avg       0.97      0.95      0.96      1689
weighted avg       0.98      0.98      0.98      1689

Confusion Matrix: [[1395   10]
 [  27  257]]


In [26]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt+=1

score = cross_val_score(ensemble.RandomForestClassifier(random_state= 42), X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

Fold:1, Train set: 4504, Test set:1126
Fold:2, Train set: 4504, Test set:1126
Fold:3, Train set: 4504, Test set:1126
Fold:4, Train set: 4504, Test set:1126
Fold:5, Train set: 4504, Test set:1126
Scores for each fold are: [0.97158082 0.95648313 0.96358792 0.97158082 0.97069272]
Average score: 0.97


In [44]:
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

catboost_model = CatBoostClassifier(verbose=False, random_state=42).fit(X_train, y_train)
y_pred = catboost_model.predict(X_test)

print(f"Accuracy: {round(accuracy_score(y_pred, y_test), 2)}")
print("="*60)
print(f"Recall: {round(recall_score(y_pred,y_test),2)}")
print("="*60)
print(f"Precision: {round(precision_score(y_pred,y_test), 2)}")
print("="*60)
print(f"F1: {round(f1_score(y_pred,y_test), 2)}")
print("="*60)
print(f"Auc: {round(roc_auc_score(y_pred,y_test), 2)}")
print("="*60)
CR = classification_report(y_test, y_pred)
CM = confusion_matrix(y_test, y_pred)
print(f"Classification Report:\n {CR}")
print("="*60)
print(f"Confusion Matrix: {CM}")
print("="*60)

Accuracy: 0.96
Recall: 0.93
Precision: 0.82
F1: 0.87
Auc: 0.95
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      1414
           1       0.93      0.82      0.87       275

    accuracy                           0.96      1689
   macro avg       0.95      0.90      0.92      1689
weighted avg       0.96      0.96      0.96      1689

Confusion Matrix: [[1396   18]
 [  49  226]]
